In [ ]:
# Convertir código de jupyter notebook + los resultados a html, pdf y word

# 1) Tienes que instalar "Quarto".
# 2) Para pdf tienes que isntalar "MinTeX".
# 3) Tu archivo Jupyter Notebook debe estar guardada.

# Estos son los comandos:

# quarto render C:/nombre_archivo_jupyternotebook.ipynb
# quarto render C:/nombre_archivo_jupyternotebook.ipynb -- to pdf
# quarto render C:/nombre_archivo_jupyternotebook.ipynb --to docx

# quarto render C:\Users\usuario\Documents\GitHub\Aprendizaje-automatico-con-python\Bloque 1 - Introducción al aprendizaje automático\PEC\M2.891_20251_PEC1-UbedaQuesadaJulio.ipynb
# quarto render "C:\Users\jubeda2\Desktop\Aprendizaje-automatico-con-python\Bloque 1 - Introducción al aprendizaje automático\PEC\M2.891_20251_PEC1-UbedaQuesadaJulio.ipynb"

# jupyter nbconvert --to html --embed-images --output "C:\Users\jubeda2\Desktop\Aprendizaje-automatico-con-python\Bloque 1 - Introducción al aprendizaje automático\PEC\M2.891_20251_PEC1-UbedaQuesadaJulio.html" "C:\Users\jubeda2\Desktop\Aprendizaje-automatico-con-python\Bloque 1 - Introducción al aprendizaje automático\PEC\M2.891_20251_PEC1-UbedaQuesadaJulio.ipynb"


# NOTA:
# Este comando debes escribirlo en la terminal.
# Debes quitar el "#" al comando,
# Debes escribir la ruta de tu archivo de código a convertir.

Me quiero quedar con la primera opcion:

Sin embargo es necesario hacer ciertos ajustes:
1.) Cambia los titulos y añadde nuevas variables
prec_cat -> Precipitacion categorizada
Mes_ano -> Mes
Nombre_dia_semana  ->
fecha -> fecha (mm/dd/yyy)
Tipo_via ->
Latitud_WGS84 y Longitud_WGS84 (Calcula una columna uniendo ambas y mira la frecuencia de ese para por tipo de gravedad en un histograma)-> Latitud y Longitud
horatmin -> Hora temperatura mínima
horatmax -> Hora temperatura máxima
horaracha -> Hora racha
horaHrMin -> Hora humedad mínima
horaHrMax -> Hora humedad máxima
2.) Las variables temporales deben ir desde la fecha mas temprana (ya sea en dias, fechas u horas) a la mas tardia.



# --- Configurar títulos de ejes ---

---
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Variables categóricas a analizar
var_cat = ['prec_cat', 'Mes_ano', 'Nombre_dia_semana', 'Hora_dia', 'fecha',
           'horatmin', 'horatmax', 'horaHrMin', 'horaHrMax', 'horaracha']

df = accidentes_meteo.copy()

# Función para crear gráficos superpuestos
def plot_categorical_by_target(df, categorical_vars, target_var='y', use_log_scale=True):
    """
    Crea gráficos de barras superpuestos para cada variable categórica,
    diferenciando por los valores de la variable objetivo.

    Parameters:
    -----------
    df : DataFrame
        DataFrame con los datos
    categorical_vars : list
        Lista de nombres de variables categóricas
    target_var : str
        Nombre de la variable objetivo (gravedad)
    use_log_scale : bool
        Si True, usa escala logarítmica en el eje Y
    """

    # Obtener las clases únicas de la variable objetivo
    target_classes = sorted(df[target_var].unique())

    # Colores para cada clase (puedes personalizarlos)
    colors = ['
#FF6B6B', '
#4ECDC4', '
#45B7D1', '
#FFA07A', '
#98D8C8']

    # Calcular número de filas y columnas para subplots
    n_vars = len(categorical_vars)
    n_cols = 2
    n_rows = (n_vars + n_cols - 1) // n_cols

    # Crear subplots
    fig = make_subplots(
        rows=n_rows, 
        cols=n_cols,
        subplot_titles=[f'{var}' for var in categorical_vars],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )

    # Iterar sobre cada variable categórica
    for idx, var in enumerate(categorical_vars):
        row = idx // n_cols + 1
        col = idx % n_cols + 1

        # Obtener las categorías únicas de la variable
        categories = df[var].value_counts().index.tolist()

        # Para cada clase de la variable objetivo
        for class_idx, target_class in enumerate(target_classes):
            # Filtrar datos por clase
            df_class = df[df[target_var] == target_class]

            # Contar frecuencias
            counts = df_class[var].value_counts().reindex(categories, fill_value=0)

            # Añadir barra
            fig.add_trace(
                go.Bar(
                    x=counts.index.astype(str),
                    y=counts.values,
                    name=f'Gravedad: {target_class}',
                    marker_color=colors[class_idx % len(colors)],
                    opacity=0.7,
                    legendgroup=f'class_{target_class}',
                    showlegend=(idx == 0),  # Mostrar leyenda solo en el primer gráfico
                ),
                row=row,
                col=col
            )

        # Configurar eje Y (escala logarítmica si se solicita)
        yaxis_type = 'log' if use_log_scale else 'linear'
        fig.update_yaxes(
            type=yaxis_type,
            title_text='Frecuencia (log)' if use_log_scale else 'Frecuencia',
            row=row,
            col=col
        )

        # Configurar eje X
        fig.update_xaxes(
            title_text=var,
            tickangle=-45,
            row=row,
            col=col
        )

    # Actualizar layout general
    fig.update_layout(
        height=400 * n_rows,
        width=1400,
        title_text='Distribución de Variables Categóricas por Gravedad',
        title_x=0.5,
        showlegend=True,
        barmode='overlay',  # Superponer barras
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        hovermode='x unified'
    )

    return fig

# Ejemplo de uso:
fig = plot_categorical_by_target(df, var_cat, target_var='gravedad', use_log_scale=True)
fig.show()